# Adjustment: detrend gated (82+83) dataset

This step applies a simple transit-masked detrend (piecewise median bins) and re-runs the most variability/systematics-sensitive diagnostics.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

sectors_used = [82, 83]
ds = sh.load_dataset()
lc_raw = sh.stitch_pdcsap_sectors(ds, sectors_used)
raw_depth_ppm, raw_depth_err_ppm = sh.estimate_depth_ppm(lc_raw)

lc_detrended = sh.detrend_transit_masked_bin_median(lc_raw, bin_hours=12.0, duration_buffer_factor=1.5, sigma_clip=5.0)
detrended_depth_ppm, detrended_depth_err_ppm = sh.estimate_depth_ppm(lc_detrended)

cand_raw = sh.make_candidate(raw_depth_ppm)
cand_det = sh.make_candidate(detrended_depth_ppm)

session_raw = sh.make_session(stitched=lc_raw, candidate=cand_raw, network=False, preset='extended')
session_det = sh.make_session(stitched=lc_detrended, candidate=cand_det, network=False, preset='extended')

out = {
  'sectors_used': sectors_used,
  'raw_depth_ppm': raw_depth_ppm,
  'raw_depth_err_ppm': raw_depth_err_ppm,
  'detrended_depth_ppm': detrended_depth_ppm,
  'detrended_depth_err_ppm': detrended_depth_err_ppm,
}
print(json.dumps(out, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output (setup)</b></summary>

```text
{
  "detrended_depth_err_ppm": 11.10228937244955,
  "detrended_depth_ppm": 215.2537784634312,
  "raw_depth_err_ppm": 11.153641839855947,
  "raw_depth_ppm": 264.57118442002604,
  "sectors_used": [
    82,
    83
  ]
}
```

</details>


In [ ]:
import json
import matplotlib.pyplot as plt

from bittr_tess_vetter.plotting import plot_phase_folded, plot_ephemeris_reliability, plot_model_comparison, plot_sweet

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='33_gated_82_83_adjust_detrend')

# Phase-folded plots
fig, ax = plt.subplots(figsize=(9, 5))
plot_phase_folded(lc_raw, cand_raw, ax=ax, bin_minutes=30.0, phase_range=(-0.25, 0.25))
ax.set_title('Raw gated 82+83: phase-folded')
fig.tight_layout()
fig.savefig(run_out_dir / 'phase_fold_raw.png', dpi=160, bbox_inches='tight')
if docs_out_dir is not None:
    fig.savefig(docs_out_dir / 'phase_fold_raw.png', dpi=160, bbox_inches='tight')
plt.show(); plt.close(fig)

fig, ax = plt.subplots(figsize=(9, 5))
plot_phase_folded(lc_detrended, cand_det, ax=ax, bin_minutes=30.0, phase_range=(-0.25, 0.25))
ax.set_title('Detrended gated 82+83: phase-folded')
fig.tight_layout()
fig.savefig(run_out_dir / 'phase_fold_detrended.png', dpi=160, bbox_inches='tight')
if docs_out_dir is not None:
    fig.savefig(docs_out_dir / 'phase_fold_detrended.png', dpi=160, bbox_inches='tight')
plt.show(); plt.close(fig)

# Re-run key checks (raw vs detrended)
r12_raw = session_raw.run('V12')
r16_raw = session_raw.run('V16')
r17_raw = session_raw.run('V17')

r12_det = session_det.run('V12')
r16_det = session_det.run('V16')
r17_det = session_det.run('V17')

# Detrended plots for V12/V16/V17
fig, ax = plt.subplots(figsize=(9, 5))
plot_sweet(r12_det, ax=ax)
ax.set_title('V12 (detrended gated 82+83): SWEET')
fig.tight_layout()
fig.savefig(run_out_dir / 'V12_sweet_detrended.png', dpi=160, bbox_inches='tight')
if docs_out_dir is not None:
    fig.savefig(docs_out_dir / 'V12_sweet_detrended.png', dpi=160, bbox_inches='tight')
plt.show(); plt.close(fig)

fig, ax = plt.subplots(figsize=(9, 5))
plot_model_comparison(r16_det, ax=ax)
ax.set_title('V16 (detrended gated 82+83): Model competition')
fig.tight_layout()
fig.savefig(run_out_dir / 'V16_model_competition_detrended.png', dpi=160, bbox_inches='tight')
if docs_out_dir is not None:
    fig.savefig(docs_out_dir / 'V16_model_competition_detrended.png', dpi=160, bbox_inches='tight')
plt.show(); plt.close(fig)

fig, ax = plt.subplots(figsize=(9, 5))
plot_ephemeris_reliability(r17_det, ax=ax)
ax.set_title('V17 (detrended gated 82+83): Ephemeris reliability')
fig.tight_layout()
fig.savefig(run_out_dir / 'V17_ephemeris_reliability_detrended.png', dpi=160, bbox_inches='tight')
if docs_out_dir is not None:
    fig.savefig(docs_out_dir / 'V17_ephemeris_reliability_detrended.png', dpi=160, bbox_inches='tight')
plt.show(); plt.close(fig)

print(json.dumps({
  'raw': {
    'V12': {'status': r12_raw.status, 'flags': list(r12_raw.flags), 'metrics': r12_raw.metrics},
    'V16': {'status': r16_raw.status, 'flags': list(r16_raw.flags), 'metrics': r16_raw.metrics},
    'V17': {'status': r17_raw.status, 'flags': list(r17_raw.flags), 'metrics': r17_raw.metrics},
  },
  'detrended': {
    'V12': {'status': r12_det.status, 'flags': list(r12_det.flags), 'metrics': r12_det.metrics},
    'V16': {'status': r16_det.status, 'flags': list(r16_det.flags), 'metrics': r16_det.metrics},
    'V17': {'status': r17_det.status, 'flags': list(r17_det.flags), 'metrics': r17_det.metrics},
  }
}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output (checks)</b></summary>

```text
{
  "detrended": {
    "V12": {
      "flags": [],
      "metrics": {
        "baseline_days": 50.98,
        "cadence_median_min": 2.0,
        "n_points": 35459,
        "n_transits_expected": 3,
        "snr_at_period": 1.5564623695532303,
        "snr_double_period": 1.134661644581126,
        "snr_half_period": 0.5947183370308814,
        "sweet_msg": "OK: SWEET finds no out-of-transit variability at transit period"
      },
      "status": "ok"
    },
    "V16": {
      "flags": [],
      "metrics": {
        "aic_eb_like": -488725.55512598663,
        "aic_transit_only": -488715.309163366,
        "aic_transit_sinusoid": -488712.17768047214,
        "artifact_prior_combined_risk": 0.2401026934306569,
        "artifact_prior_period_alias_risk": 0.4802053868613138,
        "artifact_prior_scattered_light_risk": 0.0,
        "artifact_prior_sector_quality_risk": 0.0,
        "artifact_risk": 0.5,
        "bic_eb_like": -488700.1267288513,
        "bic_transit_only": -488706.83303098753,
        "bic_transit_sinusoid": -488669.79701857996,
        "model_competition_label": "AMBIGUOUS",
        "winner": "transit_only",
        "winner_margin_bic": 6.706302136240993
      },
      "status": "ok"
    },
    "V17": {
      "flags": [],
      "metrics": {
        "depth_hat_ppm": 237.147021815065,
        "depth_sigma_ppm": 9.484080961888731,
        "effective_n_points": 379.9577353434139,
        "max_ablation_score_drop_fraction": 0.03403549499446231,
        "n_in_transit": 487,
        "null_percentile": 0.9950248756218906,
        "period_neighborhood_best_period_days": 14.23952392552,
        "period_neighborhood_best_score": 25.072061707632148,
        "period_neighborhood_second_best_score": 25.004744557540953,
        "period_peak_to_next_ratio": 1.0026921750764655,
        "phase_shift_null_p_value": 0.004975124378109453,
        "phase_shift_null_z": 8.851063165718898,
        "score": 25.004744557540953,
        "top_5_fraction_abs": 0.0
      },
      "status": "ok"
    }
  },
  "raw": {
    "V12": {
      "flags": [],
      "metrics": {
        "baseline_days": 50.98,
        "cadence_median_min": 2.0,
        "n_points": 35459,
        "n_transits_expected": 3,
        "snr_at_period": 6.901905743025918,
        "snr_double_period": 1.1145785781293491,
        "snr_half_period": 1.8373496539712335,
        "sweet_msg": "WARN: SWEET test finds signal at the transit period"
      },
      "status": "ok"
    },
    "V16": {
      "flags": [
        "MODEL_PREFERS_NON_TRANSIT"
      ],
      "metrics": {
        "aic_eb_like": -485417.13288106344,
        "aic_transit_only": -485404.0707287799,
        "aic_transit_sinusoid": -485474.502500759,
        "artifact_prior_combined_risk": 0.2401026934306569,
        "artifact_prior_period_alias_risk": 0.4802053868613138,
        "artifact_prior_scattered_light_risk": 0.0,
        "artifact_prior_sector_quality_risk": 0.0,
        "artifact_risk": 0.8,
        "bic_eb_like": -485391.7044839281,
        "bic_transit_only": -485395.59459640144,
        "bic_transit_sinusoid": -485432.1218388668,
        "model_competition_label": "SINUSOID",
        "winner": "transit_sinusoid",
        "winner_margin_bic": 36.527242465352174
      },
      "status": "ok"
    },
    "V17": {
      "flags": [],
      "metrics": {
        "depth_hat_ppm": 267.1192365173071,
        "depth_sigma_ppm": 9.483800429830776,
        "effective_n_points": 385.2553844525222,
        "max_ablation_score_drop_fraction": 0.030711572964288707,
        "n_in_transit": 487,
        "null_percentile": 0.9950248756218906,
        "period_neighborhood_best_period_days": 14.245220874480001,
        "period_neighborhood_best_score": 28.22545985045269,
        "period_neighborhood_second_best_score": 28.16584327071014,
        "period_peak_to_next_ratio": 1.0021166268366104,
        "phase_shift_null_p_value": 0.004975124378109453,
        "phase_shift_null_z": 5.3034546716866755,
        "score": 28.16584327071014,
        "top_5_fraction_abs": 0.0
      },
      "status": "ok"
    }
  }
}
```

</details>


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/phase_fold_raw.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/phase_fold_raw.png",
  "status": "ok"
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/phase_fold_raw.png" width="900" />


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/phase_fold_detrended.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/phase_fold_detrended.png",
  "status": "ok"
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/phase_fold_detrended.png" width="900" />


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/V12_sweet_detrended.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/V12_sweet_detrended.png",
  "status": "ok"
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/V12_sweet_detrended.png" width="900" />


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/V16_model_competition_detrended.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/V16_model_competition_detrended.png",
  "status": "ok"
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/V16_model_competition_detrended.png" width="900" />


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/V17_ephemeris_reliability_detrended.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/V17_ephemeris_reliability_detrended.png",
  "status": "ok"
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/33_gated_82_83_adjust_detrend/V17_ephemeris_reliability_detrended.png" width="900" />


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none (this step changes preprocessing).
- **Result:** detrending removes the SWEET warning and shifts V16’s preferred model from `transit_sinusoid` to `transit_only`.
- **Warning:** the inferred depth changes materially after detrending (raw ~265 ppm vs detrended ~215 ppm), so detrending choice is scientifically consequential here.
- **Interpretation:** this supports the hypothesis that variability/systematics were contaminating the original signal, but it also means the transit depth is not yet robust.
- **Next step:** pick a single detrending configuration (window, clipping) and re-run the full gated suite (V01–V05, V11–V21, V16–V19) on that fixed detrended dataset to check which conclusions remain stable.

</details>
